### Analysis of Liverpool 2023 Voting
This analysis assumes different voting schemes, which all have in common that all countries apart from the last get points. This scheme would work for 25 participants, too, as then the last act would get 1 point instead of 0.

First case assumes that the points from 0 - 26 are distributed evenly, while the second scheme assumes that the second best gets 26 and the best 28 points.

The data is scraped from eurovision.tv for all countries

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [2]:
base_path = 'https://eurovision.tv/event/liverpool-2023/grand-final/results/{0}'
header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
# countries = ('albania', 'armenia', 'australia', 'austria', 'belgium', 'croatia', 'cyprus', 'czechia', 'estonia', 'finland', 'france', 'germany', 'israel', 'italy', 'lithuania', 'moldova', 'norway', 'poland', 'portugal', 'serbia', 'slovenia', 'spain', 'sweden', 'switzerland', 'ukraine', 'united kingdom')
voting_type = ('juryA', 'juryB', 'juryC', 'juryD', 'juryE')
load_from_internet = True

Reading out the names of the voting countries from the list of countries entry

In [3]:

respones_for_countries = requests.get(base_path.format('albania'), headers=header)
countries_soup = BeautifulSoup(respones_for_countries.text, 'html.parser')


In [4]:
countries = []
country_selection = countries_soup.find('select', class_='form-select')
for con in country_selection.children:
    countries.append(con.text.lower().replace(' ', '-'))

First, lets create the final data structure to be populated. The data structure contains a line for each country, and the columns are the origins of the points.
The origins are named by the following scheme: '<country>_tele', '<country>_juryA', etc

In [5]:
columns_jury = []
columns_jury_final = []
columns_tele = []
for voted_to_coun in countries:
    for vote in voting_type:
        columns_jury.append(f'{voted_to_coun}_{vote}')
    columns_tele.append(f'{voted_to_coun}_tele')
    columns_jury_final.append(f'{voted_to_coun}_jury')
        
votes_jury = pd.DataFrame(index=countries, columns=columns_jury)
votes_tele = pd.DataFrame(index=countries, columns=columns_tele)

In [6]:
if load_from_internet:
    for voted_to_coun in countries:
        voted_to_coun = voted_to_coun.replace(' ', '-')
        print(base_path.format(voted_to_coun))
        response = requests.get(base_path.format(voted_to_coun), headers=header)
        soup = BeautifulSoup(response.text, 'html.parser')
        detailed_voting = soup.find('table').find('tbody')
        rows = detailed_voting.findAll('tr')
        for row in rows:
            cols = row.findAll('td')
            origin = cols[0].text.strip().lower().replace(' ', '-')
            # Add jury votes
            votes_jury.loc[origin, f'{voted_to_coun}_juryA'] = int(cols[2].text.strip())
            votes_jury.loc[origin, f'{voted_to_coun}_juryB'] = int(cols[3].text.strip())
            votes_jury.loc[origin, f'{voted_to_coun}_juryC'] = int(cols[4].text.strip())
            votes_jury.loc[origin, f'{voted_to_coun}_juryD'] = int(cols[5].text.strip())
            if len(cols) == 9:
                votes_jury.loc[origin, f'{voted_to_coun}_juryE'] = int(cols[6].text.strip())
            # Add televotes (more complicated due to structure)
            votes_tele.loc[origin, f'{voted_to_coun}_tele'] = int(cols[-1].text.split(' ')[-1].strip('stndrh'))
else:
    votes_jury = pd.read_csv(r'voting_results_table_jury.csv')
    votes_tele = pd.read_csv(r'voting_results_table_tele.csv')

https://eurovision.tv/event/liverpool-2023/grand-final/results/albania
https://eurovision.tv/event/liverpool-2023/grand-final/results/armenia
https://eurovision.tv/event/liverpool-2023/grand-final/results/australia
https://eurovision.tv/event/liverpool-2023/grand-final/results/austria
https://eurovision.tv/event/liverpool-2023/grand-final/results/azerbaijan
https://eurovision.tv/event/liverpool-2023/grand-final/results/belgium
https://eurovision.tv/event/liverpool-2023/grand-final/results/croatia
https://eurovision.tv/event/liverpool-2023/grand-final/results/cyprus
https://eurovision.tv/event/liverpool-2023/grand-final/results/czechia
https://eurovision.tv/event/liverpool-2023/grand-final/results/denmark
https://eurovision.tv/event/liverpool-2023/grand-final/results/estonia
https://eurovision.tv/event/liverpool-2023/grand-final/results/finland
https://eurovision.tv/event/liverpool-2023/grand-final/results/france
https://eurovision.tv/event/liverpool-2023/grand-final/results/georgia
htt

In [7]:
# save data frame to file to not depend on internet connection
votes_jury.to_csv(r'voting_results_table_jury.csv')
votes_tele.to_csv(r'voting_results_table_tele.csv')

Combining jury votes per country, ordering them from lowest to highest rank, and reassigning 1st to 26th place (26th place will always be the country itself getting 0 points from itself)

### Implement voting schema

#### First trying to reproduce the official voting scheme

This is not a piece of cake, as the voting system is maximally intransparent... See e.g. https://thateurovisionsite.com/2023/04/12/eurovision-2023-voting-rules/

Each jury member ranks from 1-25, this is converted to points from 12 decreasing in the usual manner. These points from all judges are combined and then sorted again, and the first 10 get the usual points.

In [8]:
votes_jury_combined = pd.DataFrame(index=countries, columns=columns_jury_final)

In [13]:
jury_points = np.zeros((26,))
jury_points[0:10] = np.array([12, 10, 8, 7, 6, 5, 4, 3, 2, 1])
ranking = np.array([i + 1 for i in range(26)])

In [15]:
votes_jury_points_individual = votes_jury.replace(ranking, jury_points)

In [16]:
votes_jury_individual_control = votes_jury_points_individual.sum(axis=0)

In [17]:
for voting_coun in countries:
    for voted_to_coun in countries:
        votes_jury_combined.loc[voting_coun, f'{voted_to_coun}_jury'] = votes_jury_points_individual.loc[voting_coun, (col for col in votes_jury if col.startswith(f'{voted_to_coun}'))].sum()

Convert the placing into a placing from 1st to last

In [19]:
votes_jury_ranking = pd.DataFrame(0, index=countries, columns=columns_jury_final)

for voting_coun in countries:
    votes_jury_ranking.loc[:, f'{voting_coun}_jury'] = votes_jury_combined.loc[:, f'{voting_coun}_jury'].rank(ascending=False, method='min').replace(ranking, jury_points)

In [21]:
jury_votes_total = votes_jury_ranking.sum(axis=1)